[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/claresinger/PySDM-examples/blob/ese134/PySDM_examples/Pyrcel/condensation_playground.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/claresinger/PySDM-examples/blob/ese134/PySDM_examples/Pyrcel/condensation_playground.ipynb)

#### based on Example Figure from Pyrcel code documentation https://pyrcel.readthedocs.io/en/latest/examples/basic_run.html

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install "atmos-cloud-sim-uj-utils
    from atmos_cloud_sim_uj_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [2]:
import numpy as np
from matplotlib import pyplot
from scipy.ndimage.filters import uniform_filter1d
from atmos_cloud_sim_uj_utils import show_plot

from PySDM import Formulae
from PySDM.physics import si
from PySDM.initialisation.spectra import Lognormal
from PySDM.products import (
    ParcelDisplacement, AmbientTemperature, AmbientRelativeHumidity,
    ParticleSizeSpectrumPerVolume, ParticleVolumeVersusRadiusLogarithmSpectrum
)

from PySDM_examples.Pyrcel import Settings, Simulation

from numpy import errstate
from IPython.display import display
from PySDM_examples.Pyrcel.profile_plotter import ProfilePlotter
from PySDM_examples.utils import widgets

In [3]:
progbar = widgets.IntProgress(min=0, max=100, description='%')

In [19]:
def demo(*, freezer, k2, N2, r2):
    with freezer:
        with errstate(all='raise'):
            settings = Settings(
                dz = 1 * si.m,
                n_sd_per_mode = (10, 10),
                aerosol_modes_by_kappa = {
                    .54: Lognormal(
                        norm_factor=850 / si.cm ** 3,
                        m_mode=15 * si.nm,
                        s_geom=1.6
                    ),
                    k2: Lognormal(
                        norm_factor=N2 / si.cm ** 3,
                        m_mode=r2 * si.nm,
                        s_geom=1.2
                    )
                },
                vertical_velocity = 1.0 * si.m / si.s,
                initial_pressure = 775 * si.mbar,
                initial_temperature = 274 * si.K,
                initial_relative_humidity = 0.98,
                displacement = 250 * si.m,
                formulae = Formulae(constants={'MAC': .3})
            )
            dry_radius_bin_edges = np.logspace(
                np.log10(1e-3 * si.um),
                np.log10(5e0 * si.um),
                33, endpoint=False
            )
            simulation = Simulation(
                settings,
                products=(
                ParcelDisplacement(
                    name='z'),
                AmbientRelativeHumidity(
                    name='S_max', unit='%', var='RH'),
                AmbientTemperature(
                    name='T'),
                ParticleSizeSpectrumPerVolume(
                    name='dry:dN/dR', radius_bins_edges=dry_radius_bin_edges, dry=True),
                ParticleVolumeVersusRadiusLogarithmSpectrum(
                    name='dry:dV/dlnR', radius_bins_edges=dry_radius_bin_edges, dry=True),
                ),
                scipy_solver=False,
            )
            output = simulation.run((widgets.ProgbarUpdater(progbar, settings.output_steps[-1]),))

        with errstate(invalid='ignore'):
            plotter = ProfilePlotter(settings)
            plotter.plot(output)
            plotter.show()
            
            

In [29]:
k2 = widgets.FloatSlider(value=1.2, min=0.2, max=1.4, step=0.1, description='κ2', continuous_update=False, readout_format='.1f')
N2 = widgets.IntSlider(value=10, min=5, max=50, step=5, description='N2', continuous_update=False)
r2 = widgets.IntSlider(value=850, min=200, max=1000, step=50, description='r2', continuous_update=False)
sliders = widgets.HBox([k2, N2, r2])
freezer = widgets.Freezer([k2, N2, r2])
self = widgets.interactive_output(demo, {'freezer': freezer, 'k2': k2, 'N2': N2, 'r2': r2})

display(sliders, progbar, self)

IntProgress(value=100, description='%')

Output()